In [11]:
import os
import warnings
warnings.filterwarnings("ignore")

In [12]:
import pandas as pd
import chromadb
import os
import csv
from chromadb.utils import embedding_functions

In [13]:
INFO_CSV = "./Dataset/patent_dataset_cpc.csv"    # 특허 정보 파일 경로
CLAIMS_CSV = "./Dataset/claim_dataset_cpc.csv"  # 특허 청구항 파일 경로
DB_DIR = "./patent_chroma_db"     # ChromaDB가 저장될 디렉토리
COLLECTION_NAME = "patents"     # ChromaDB 컬렉션 이름

In [30]:
# --- 4. 추론 실행 ---
# 서비스 설명 및 출력 통제 지시사항을 포함한 시스템 프롬프트 구성
#추가 프롬: - 반드시 JSON 포맷으로 응답

## 평가자 llm 시스템 프롬프트
router_system_prompt = """
당신은 사용자가 아이디어를 구체화하고 유사 특허를 검색할 수 있도록 돕는 'AI 특허 전략가'입니다.

당신의 임무는 사용자의 [초기 아이디어]를 분석하여 다음 두 가지 시나리오 중 하나로 행동하는 것입니다.

---
### 📜 시나리오 1: 아이디어가 '너무 광범위한' 경우
(판단 기준: 단순 명사 나열, 해결하려는 구체적 문제 부재, 기술적 수단 불명확)

만약 아이디어가 "자동차", "AI 서비스", "배달 드론" 같이 너무 일반적이라면, **절대 검색 함수를 호출하지 마십시오.** 대신 사용자가 아이디어를 구체화하도록 유도해야 합니다.

**[응답 가이드]**
1. **문제 지적:** 현재 키워드로는 검색 범위가 너무 넓음을 부드럽게 설명합니다.
2. **탐색 질문:** '해결하려는 문제'나 '차별화된 기술적 수단'이 무엇인지 묻습니다.
3. **예시 제안:** 사용자의 입력과 관련된 구체적인 하위 기술 키워드 3~4개를 제안합니다.

---
### 📜 시나리오 2: 아이디어가 '충분히 구체적인' 경우
(판단 기준: [기술적 수단] + [해결 과제] 또는 [구체적 응용 분야]가 명시됨)

아이디어가 구체적이라고 판단되면, 사용자에게 칭찬의 말을 건네고 **즉시 `search_chunks` 함수를 호출**하십시오.
이때, `query` 인자는 사용자의 입력을 그대로 쓰지 말고 아래 규칙에 따라 **'특허 검색 최적화 문장'**으로 재작성해야 합니다.

**[검색 쿼리 변환 규칙 (매우 중요)]**
1.  **단순 키워드 나열 금지:** "손잡이 선풍기 유모차" 처럼 명사만 나열하지 마십시오. 관계성이 사라져 검색 품질이 떨어집니다.
2.  **기술적 서술문 작성:** 특허의 **[발명의 명칭]**이나 **[요약]** 처럼, 구성 요소 간의 **결합 관계**와 **기능**이 드러나는 문장으로 변환하십시오.
3.  **필수 포함 요소:**
    * **대상:** (예: 유모차)
    * **구성:** (예: 핸들에 착탈식으로 결합되는 송풍 유닛)
    * **목적/효과:** (예: 보호자의 쾌적함 제공)

**[변환 예시]**
* *사용자:* "손잡이에 선풍기를 달아서 산모들이 시원한 유모차"
* *Query:* **"보호자 냉방을 위해 핸들 프레임에 결합된 송풍 장치를 구비한 유모차 시스템"**

**[함수 호출 형식 (JSON)]**
당신은 텍스트 응답을 마친 후, 반드시 아래의 'search_query' 도구를 호출해야 합니다.

tool": "search_query",
arguments: query(변환된_검색_최적화_쿼리)

"""

evaluation_system_prompt = """
당신은 AI 검색 결과를 검증하는 '최종 특허 심사관'입니다.

[특허 문서 조각]은 1차 AI 검색(RAG)을 통해 [사용자 아이디어]와 '의미상' 유사할 가능성이 있다고 **후보로** 검색된 결과입니다.

당신의 임무는 이 1차 검색 결과를 맹신하지 말고, [사용자 아이디어]의 **'기술적 핵심 원리'** 관점에서 **전문가로서 2차 검증(Evaluation)**을 수행하는 것입니다. 1차 검색이 표면적인 키워드나 문맥에 속았을 수 있으니(예: 'AI 진단'은 같으나 '피부' vs '자동차 도색'처럼 분야가 다름), 이를 비판적으로 분석해야 합니다.

당신의 응답은 `cal_evalscore` 함수를 호출하는 것이어야 합니다.

---
### ⚖️ 평가 지침

1.  **`eval_score` (0-100점 사이의 정수):**
    * 이 점수는 [사용자 아이디어]와 [특허 조각]의 **'기술적 핵심 원리의 일치도'**입니다.
    * 아래 벤치마크를 **참고**하여 구체적인 정수 점수를 도출하세요.

    * **0점 (기준):** 1차 검색이 명백히 오류임. 키워드만 겹칠 뿐, 기술 분야와 원리가 완전히 무관합니다.
    * **25점 (참고 기준):** 1차 검색이 '의미'는 맞췄으나(예: 'AI 진단'), **'기술 분야(Domain)'**가 다릅니다. (예: 아이디어는 '의료 피부'인데 특허는 '자동차 도색')
    * **50점 (중간 기준):** 기술 분야와 해결하려는 문제는 유사하나, **'핵심 기술 수단(접근 방식)'**이 상이합니다.
    * **75점 (참고 기준):** 핵심 기술 원리 및 접근 방식이 매우 유사하며, 일부 부수적인 구성 요소에서만 차이를 보입니다.
    * **100점 (기준):** 두 내용의 핵심 기술적 사상이 실질적으로 동일합니다.

2.  **`reason` (문자열):**
    * **[매우 중요]** 이 점수를 준 '이유'를 명확히 설명해야 합니다.
    * **(점수가 낮을 경우):** "1차 검색에서는 유사하게 보였으나, [아이디어]의 '핵심 분야(예: 의료)'와 달리 이 특허는 '(예: 자동차)' 분야에 관한 것이므로 점수를 낮게 책정했습니다."와 같이, RAG가 왜 이것을 가져왔는지 + 그럼에도 왜 점수가 낮은지를 함께 설명해야 합니다.
    * **(점수가 높을 경우):** [아이디어]의 '[X] 개념'이 특허의 '[Y] 부분'과 어떻게 기술적으로 일치하는지 설명합니다.
---

이제 [사용자 아이디어]와 [특허 문서 조각]을 바탕으로 분석을 수행하고, 즉시 `cal_evalscore` 함수를 호출하세요.
"""


In [35]:
# --- 5. Function Calling 테스트 ---
search_tools = [
    {
        "type": "function",
        "function": {
            "name": "search_query",
            "description": "사용자의 아이디어가 구체적일 때, 관련 특허 문서를 검색하기 위해 호출합니다. RAG 시스템을 통해 의미적으로 유사한 특허를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "RAG 검색을 위한 기술적 서술문입니다. 단순 키워드 나열(예: 'A B C')을 절대 금지합니다. 대신 'A의 기능을 수행하기 위해 B에 결합된 C 장치'와 같이 구성 요소 간의 관계와 목적이 명확한 문장 형태(특허 명칭 스타일)로 입력해야 합니다."
                    }
                },
                "required": ["query"],
            },
        },
    }
]


# 5-2. 도구(함수) 목록 정의 (OpenAI tool-call 형식)
eval_tools = [
    {
        "type": "function",
        "function": {
            "name": "cal_evalscore",
            "description": "[사용자 아이디어]와 [특허 문서 조각]의 유사도를 분석하여, 0-100점 사이의 점수와 그 근거를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "eval_score": {
                        "type": "integer",
                        "description": "[사용자 아이디어]와 [특허 문서 조각] 간의 기술적 유사도 점수. 0 (완전히 무관함)에서 100 (기술적으로 동일함) 사이의 정수입니다.",
                        "minimum": 0,
                        "maximum": 100
                    },
                    "reason": {
                        "type": "string",
                        "description": "해당 점수를 부여한 구체적인 이유. 특허 조각의 어느 부분이 아이디어의 어떤 개념과 유사한지(또는 다른지) 명확히 짚어서 설명해야 합니다."
                    }
                },
                "required": ["eval_score", "reason"],
            },
        },
    }
]

TOOL_MAPPING = {"search_query": search_query}



In [29]:
def get_unique_patents(results):
    """
    ChromaDB 검색 결과에서 청구 번호(ApplicationNumber) 기준으로 중복을 제거하고,
    각 특허별 가장 유사도가 높은(distance가 낮은) 청크만 남겨 상위 k개를 반환합니다.
    """
    
    # 1. 중복 제거를 위한 딕셔너리 (Key: 청구번호, Value: 해당 특허의 베스트 청크 정보)
    unique_patents = {}
    
    # 검색된 결과의 개수만큼 반복
    num_results = len(results['documents'][0])
    
    for i in range(num_results):
        # 정보 추출
        metadata = results['metadatas'][0][i]
        document = results['documents'][0][i]
        distance = results['distances'][0][i] # 코사인 유사도 거리 (낮을수록 유사함)
        
        # 청구 번호 추출 (그룹화의 기준 Key)
        app_number = metadata.get('ApplicationNumber')
        
        # 예외 처리: 청구 번호가 없는 경우 스킵 (데이터 무결성 체크)
        if not app_number:
            continue
            
        # 2. 그룹화 및 최적 청크 선별 로직
        if app_number not in unique_patents:
            # (A) 처음 발견된 특허라면 -> 딕셔너리에 저장
            unique_patents[app_number] = {
                "metadata": metadata,
                "document": document,
                "distance": distance
            }
        else:
            # (B) 이미 발견된 특허라면 -> 더 유사한지(distance가 더 작은지) 비교
            existing_distance = unique_patents[app_number]['distance']
            
            if distance < existing_distance:
                # 현재 청크가 기존 청크보다 더 유사하다면 정보 갱신
                unique_patents[app_number] = {
                    "metadata": metadata,
                    "document": document,
                    "distance": distance
                }
    
    # 3. 딕셔너리를 리스트로 변환
    unique_list = list(unique_patents.values())
    
    # 4. 거리(distance) 기준으로 오름차순 정렬 (낮은게 1등)
    unique_list.sort(key=lambda x: x['distance'])
    
    # 5. 사용자가 원하는 개수(target_k)만큼 자르기
    #final_results = unique_list[:target_k]
    final_results = unique_list
    
    return final_results

def search_query(query, db_path="./patent_chroma_db", collection_name="patents", model_name="nlpai-lab/KURE-v1", n_results=20):
    """
    지정된 ChromaDB에서 아이디어(쿼리 텍스트)를 검색합니다.
    """
    query_text = query
    print(f"\n--- 테스트 검색 시작 ---")
    print(f"Query: '{query_text}'")
    
    try:
        # 1. DB 클라이언트 초기화
        client = chromadb.PersistentClient(path=db_path)
        
        # 2. 임베딩 함수 설정 (DB에 저장할 때 사용한 것과 동일해야 함)
        try:
            embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
                model_name=model_name
            )
        except Exception as e:
            print(f"검색을 위한 임베딩 모델 로드 중 오류 발생: {e}")
            return
        
        # 3. 컬렉션 가져오기 (get_collection 사용)
        try:
            collection = client.get_collection(
                name=collection_name,
                embedding_function=embedding_func
            )
            print(f"'{collection_name}' 컬렉션 (문서 {collection.count()}개)을 성공적으로 불러왔습니다.")
        except Exception as e:
            print(f"'{collection_name}' 컬렉션 가져오기 중 오류 발생: {e}")
            print("'process_patents_to_chroma' 함수가 먼저 성공적으로 실행되었는지 확인하세요.")
            return

        # 4. 쿼리 실행
        results = collection.query(
            query_texts=[query_text],
            n_results=n_results,
            include=["metadatas", "documents", "distances"] # 거리(유사도)도 포함
        )
        
        print(f"\n--- 검색 결과 (상위 {len(results.get('ids', [[]])[0])}개) ---")
        
        # 5. 결과 출력
        if not results or not results.get('ids', [[]])[0]:
            print("검색 결과가 없습니다.")
            return
        results = get_unique_patents(results) #중복 특허 제거

        return results
        
        for idx, item in enumerate(results):
            distance = item['distance']
            metadata = item['metadata']
            document = item['document']
            
            print(f"\n[Result {idx+1}]")
            print(f"  Distance  : {distance:.4f} (낮을수록 유사)")
            print(f"  InventionName    : {metadata.get('InventionName')}")
            print(f"  Applicant    : {metadata.get('Applicant')}")
            print(f"  ApplicationNumber: {metadata.get('ApplicationNumber')}")
            print(f"  Document  : {document[:250]}...") # 문서가 길 수 있으므로 250자만 미리보기
            print("-" * 20)
            
    except Exception as e:
        print(f"검색 중 예상치 못한 오류가 발생했습니다: {e}")

In [38]:
def evaluation_idea(user_idea: str, patent_chunk: str, model_name: str = "x-ai/grok-4.1-fast"):
    print(f"[사용자 아이디어]: {user_idea}")
    print(f"[특허 문서 조각]: {patent_chunk[:100]}")

    user_query = f"[사용자 아이디어]: {user_idea}\n\n[특허 문서 조각]: {patent_chunk}"
    
    messages = [
  {
    "role": "system",
    "content": evaluation_system_prompt
  },
  {
    "role": "user",
    "content": user_query,
  }
]
    request = {
    "model": model_name,
    "tools": search_tools,
    "messages": messages
}
    try:
        # 1. '아이디어 게이트키퍼' LLM 호출
        response = openai_client.chat.completions.create(**request)


        print("\n[게이트키퍼 LLM 응답]")
        print("--------------------")
        print("--------------------")
        print(response.choices[0].message.content)
        
        for tool_call in response.choices[0].message.tool_calls:

            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

        eval_result = [tool_args["eval_score"], tool_args["reason"]]
        return eval_result
        

    except Exception as e:
        print(f"\n--- [오류] LLM API 호출 또는 라우팅 중 오류 발생 ---")
        print(f"에러 상세: {e}")
        return {"status": "error", "message": str(e)}
    return 'sex'

In [25]:
test_query = "삭제" 
    
search_query(test_query)


--- 테스트 검색 시작 ---


NameError: name 'query_text' is not defined

In [ ]:
#llm 메인 모델 설정

In [32]:
import json, requests
from openai import OpenAI

OPENROUTER_API_KEY = ""

openai_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

def execute_router(user_query: str, model_name: str = "x-ai/grok-4.1-fast"):
    """
    사용자 아이디어를 받아 게이트키퍼 LLM을 호출하고,
    결과에 따라 RAG 검색을 트리거하거나 사용자에게 피드백을 반환합니다.
    """

    print(f"\n--- [EXECUTE ROUTER] ---")
    print(f"입력 아이디어: '{user_query}'")

    messages = [
  {
    "role": "system",
    "content": router_system_prompt
  },
  {
    "role": "user",
    "content": user_query,
  }
]
    request = {
    "model": model_name,
    "tools": search_tools,
    "messages": messages
}
    try:
        # 1. '아이디어 게이트키퍼' LLM 호출
        response = openai_client.chat.completions.create(**request)


        print("\n[게이트키퍼 LLM 응답]")
        print("--------------------")
        print("--------------------")
        print(response.choices[0].message.content)

        for tool_call in response.choices[0].message.tool_calls:

            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

        patent_chunks = TOOL_MAPPING[tool_name](**tool_args)
        for idx, item in enumerate(patent_chunks):
            document = item['document']
            eval_result = evaluation_idea(user_query, document)

    except Exception as e:
        print(f"\n--- [오류] LLM API 호출 또는 라우팅 중 오류 발생 ---")
        print(f"에러 상세: {e}")
        return {"status": "error", "message": str(e)}
        


In [33]:
execute_router("PRS의 전송에 사용하기 위해, 상기 SL BWP 또는 상기 측정 갭 구간을 선택하는 단계")


--- [EXECUTE ROUTER] ---
입력 아이디어: 'PRS의 전송에 사용하기 위해, 상기 SL BWP 또는 상기 측정 갭 구간을 선택하는 단계'

[게이트키퍼 LLM 응답]
--------------------
--------------------
사용자의 아이디어는 PRS(위치 기준 신호) 전송을 위한 SL BWP(사이드링크 대역폭 부분) 또는 측정 갭 구간 선택이라는 구체적인 기술적 수단과 목적이 잘 드러나 매우 훌륭합니다. 이를 바탕으로 유사 특허를 검색해 보겠습니다.



--- 테스트 검색 시작 ---
Query: 'PRS 신호의 전송을 위해 SL BWP 또는 측정 갭 구간을 선택하는 단계를 포함하는 무선 통신 시스템 및 방법'
'patents' 컬렉션 (문서 119개)을 성공적으로 불러왔습니다.

--- 검색 결과 (상위 20개) ---

[Result 1]
  Distance  : 0.4013 (낮을수록 유사)
  InventionName    : 무선 통신 시스템에서 데이터 휴지 구간 예측 방법 및 장치
  Applicant    : 삼성전자주식회사
  ApplicationNumber: 1020240060334
  Document  : 특허명: 무선 통신 시스템에서 데이터 휴지 구간 예측 방법 및 장치

요약: 본 개시는 보다 높은 데이터 전송률을 지원하기 위한 5G 또는 6G 통신 시스템에 관련된 것이다. 본 개시의 무선 통신 시스템에서 제어 신호 처리 방법은, 기지국으로부터 전송되는 제1 제어 신호를 수신하는 단계, 수신된 제1 제어 신호를 처리하는 단계, 처리에 기반하여 생성된 제2 제어 신호를 기지국으로 전송하는 단계를 포함하는 것을 특징으로 할 수 있다.

청구항: ...
--------------------

[Result 2]
  Distance  : 0.4268 (낮을수록 유사)
  InventionName    : 다수의 유심(USIM)을 구비한 WTRU에 대한 페이징 방법
  Applicant    : 인터디지탈 패

In [ ]:
def answer_with_rag(user_query: str):
    """
    사용자 질문에 대해 RAG를 통해 찾은 정보와 함께 LLM에게 답변을 요청하는 함수
    """
    print(f"\n--- 사용자 질문: {user_query} ---")
    
    # 1. Retrieve: RAG Retriever로 사용자 질문과 가장 관련 높은 문서를 검색합니다.
    retrieved_docs = rag_retriever.invoke(user_query)
    print(f"📚 {len(retrieved_docs)}개의 관련 문서를 찾았습니다.")

    # 2. 컨텍스트(Context) 생성: 검색된 문서들의 내용을 하나의 문자열로 합칩니다.
    context_data = "\n".join([f"- {doc.page_content}" for doc in retrieved_docs])
    '''
    print("--- 검색된 RAG 데이터 ---")
    print(context_data)
    print("-------------------------")
    '''
    # 3. 프롬프트 구성: 기존의 프롬프트 형식에 질문과 RAG 데이터를 결합합니다.
    # 이는 get_single_turn_response 함수 내부에서 처리되므로,
    # 해당 함수에 전달할 형태로 질문을 재구성합니다.
    final_query_for_llm = f"""
    아래 [데이터]를 참고하여 질문에 대답해주세요.
    
    [질문]
    {user_query}

    [데이터]
    {context_data}
    """
    
    # 4. Generate: 재구성된 질문을 기존에 만들었던 LLM 호출 함수에 전달합니다.
    final_answer = get_single_turn_response(final_query_for_llm)
    
    return final_answer

In [ ]:
# --- 실제 서비스 로직 예시 (Stateless) ---

# Case 1: 일반적인 파싱
print("--- Case 1: 함수 호출 질문 ---")
user_input_1 = ""
final_response_1 = answer_with_rag(user_input_1)
print("\n[최종 답변]")
print(final_response_1)